# This notebooks shows proof of concept of Behaiv

### 0. Project idea
In this POC we'll use idea of score tracking e.g Todoist  
Our client is a sportsmen and engineer.

Person will have different tracking lists:
* Sport
* Work
* Food
* Self-improvement

Client's schedule:
* 5:00 - wake up
* 5:30 - Coffee time
* 6:00-7:30 - Gym
* 7:30 - Meditation
* 8:00-18:00 work
* 18:30 - Meditation
* 19:00 - Jogging
* 20:00 - Food
* 21:00 - Sleep

Idea is to construct model so that it'll fit his own targets.  
### 1.Gathering data
In order to get enough data we'll be using modified version of [Chain of Responsibility Pattern](https://www.tutorialspoint.com/design_pattern/chain_of_responsibility_pattern.htm)
In this case we'll simulate opened screens + external factors such as 


In [66]:

from datetime import time
import numpy as np
import pandas as pd
from sklearn import linear_model

HOME = [48.871591, 2.310870]
GYM = [48.867474, 2.333320]
WORK = [48.869740, 2.309049]
JOG = [48.869266, 2.311993]

# Simulates opening of app
features = [
    #[open time, latitude, longitude, headset_on]
    [time(5,00), HOME[0], HOME[1], 0],
    [time(5,10), HOME[0], HOME[1], 0],
    [time(6,10), GYM[0], GYM[1], 1],
    [time(7,30), HOME[0], HOME[1], 1],   
    [time(8,30), WORK[0], WORK[1], 0],
    [time(10,30), WORK[0], WORK[1], 1],
    [time(11,30), WORK[0], WORK[1], 1],
    [time(16,30), WORK[0], WORK[1], 0],
    [time(17,10), WORK[0], WORK[1], 0],
    [time(18,50), WORK[0], WORK[1], 0],
    [time(19,5), JOG[0], JOG[1], 1],
    [time(19,10), JOG[0],JOG[1], 1],
    [time(19,25), JOG[0],JOG[1], 1],
    [time(21,00), HOME[0],HOME[1], 0]    
]

features = pd.DataFrame(features, columns=["Open time", "latitude", "longitude","headset is on"])
features['Open time'] = features['Open time'].apply(lambda t: (t.hour * 60) + t.minute)

features.head()

,Open time,latitude,longitude,headset is on
0,300,48.871591,2.310870,0
1,310,48.871591,2.310870,0
2,370,48.867474,2.333320,1
3,450,48.871591,2.310870,1
4,510,48.869740,2.309049,0


In [67]:
SPORT_SCREEN = 'Sport Screen'
WORK_SCREEN = 'Work Screen'
FOOD_SCREEN = 'Food Screen'
SELFIMPROVEMENT_SCREEN = 'Self-improvement Screen'
ADD_SCREEN = 'Add new task Screen'

labels = [SELFIMPROVEMENT_SCREEN,
          SELFIMPROVEMENT_SCREEN,
          SPORT_SCREEN,
          SELFIMPROVEMENT_SCREEN,
          WORK_SCREEN,
          WORK_SCREEN,
          WORK_SCREEN,
          WORK_SCREEN,
          WORK_SCREEN,
          WORK_SCREEN,
          SPORT_SCREEN,
          SPORT_SCREEN,
          SPORT_SCREEN,
          ADD_SCREEN
          ]
#one hot encoding
labels = pd.get_dummies(labels)

label_names = labels.columns.values
labels.head()

,Add new task Screen,Self-improvement Screen,Sport Screen,Work Screen
0,0,1,0,0
1,0,1,0,0
2,0,0,1,0
3,0,1,0,0
4,0,0,0,1


In [68]:
ols = linear_model.LinearRegression()
model = ols.fit(features, labels)

test_feature = [
    [time(15,00), WORK[0], WORK[1], 0]
]

test_feature = pd.DataFrame(test_feature, columns=["Open time", "latitude", "longitude","headset is on"])
test_feature['Open time'] = test_feature['Open time'].apply(lambda t: (t.hour * 60) + t.minute)


# What will we open at 15:00 at work without headphones?
result = model.predict(test_feature)
pd.DataFrame(result, columns=label_names)


,Add new task Screen,Self-improvement Screen,Sport Screen,Work Screen
0,0.017265,-0.010301,0.061578,0.931458


As expected, when we're at work we predict that user will open work screen

Let's try another example 

In [69]:

# we use different position(simulate running)
test_feature = [
    [time(19,15), 48.867939, 2.316413, 1]
]

test_feature = pd.DataFrame(test_feature, columns=["Open time", "latitude", "longitude","headset is on"])
test_feature['Open time'] = test_feature['Open time'].apply(lambda t: (t.hour * 60) + t.minute)


# What will we open at 19:15 at JOG place with headphones?
result = model.predict(test_feature)
pd.DataFrame(result, columns=label_names)


,Add new task Screen,Self-improvement Screen,Sport Screen,Work Screen
0,-0.035641,-0.451777,0.906539,0.58088


Just as expected, even when we have different position we should open sport screen
Let's try one more example 

In [71]:
test_feature = [
    [time(21,10), HOME[0], HOME[1], 0]
]

test_feature = pd.DataFrame(test_feature, columns=["Open time", "latitude", "longitude","headset is on"])
test_feature['Open time'] = test_feature['Open time'].apply(lambda t: (t.hour * 60) + t.minute)


# What will we open at 21:10 at HOME place without headphones?
result = model.predict(test_feature)
pd.DataFrame(result, columns=label_names)

,Add new task Screen,Self-improvement Screen,Sport Screen,Work Screen
0,0.672095,0.356278,0.250886,-0.279259


As expected, we should open add new task screen at the end of a day

### Final notes
This model is just a proof of concept, optimisation and code quality is not as expected in final on-device model. 
Expected to have more features and more examples. But even with 14 examples we can build working 
model that can be used later on.

Sadly headphones param has large bias so if we put it to true it will increase Sport Screen size.
